# Convert 2016-2019 PRAMS pdf to csv

In [ ]:
import os
import warnings

import camelot
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

In [ ]:
def pdf_to_df(filename):
    df = pd.DataFrame(data = {})
    data = camelot.read_pdf(filename, flavor='stream', pages='all', strip_text='\n§#¶¥*†‡◊‖±^')
    for ii in range(2):  # assuming only two pages
        table = data[ii]
        data_dict = get_data_dict(table.df)
        df = pd.concat([df, pd.DataFrame(data_dict)])
    return df

def get_data_dict(df):
    data = {
        'Category': [],
        'Indicator': [],
        'SubIndicator': [],
        'Year': [],
        'SampleSize': [],
        'Prevalence': [],
        'CI': []
    }
    for ii in range(len(df)):
        if is_category(ii, df):
            category = format_chars(df.iloc[ii, 0])
        if is_year(ii, df):
            year_dict = get_year_dict(ii, df)
        if is_indicator(ii, df):
            indicator = format_chars(df.iloc[ii, 1])
            if is_data(ii, df):  # data row
                add_row(ii, df, data, category, indicator, year_dict)
            else:
                if is_subindicator(ii + 1, df):  # subindicator row
                    jj = ii + 1
                    while is_subindicator(jj, df):
                        sub_indicator = format_chars(df.iloc[jj, 1].split('\n')[0].rstrip())
                        add_row(jj, df, data, category, indicator, year_dict, sub_indicator)
                        jj += 1
                else:  # multiline row
                    indicator = indicator.rstrip() + ' ' + df.iloc[ii + 2, 1]
                    add_row(ii + 1, df, data, category, indicator, year_dict)     
    return data

def add_row(ii, df, data, category, indicator, year_dict, sub_indicator=np.nan):
    if is_data(ii, df):
        for year in year_dict:
            stats = df.iloc[ii, year_dict[year]]
            if stats != '':  # only append years with data
                data['Category'].append(category)
                data['Indicator'].append(indicator)
                data['SubIndicator'].append(sub_indicator)
                data['Year'].append(year)
                data['SampleSize'].append(df.iloc[ii, year_dict[year] - 1])
                data['Prevalence'].append(stats.split()[0])
                data['CI'].append(stats.split()[1])
    else:
        raise ValueError(f"Row {ii} does not contain data.")

def is_category(ii, df):
    if ii >= len(df) - 1:
        return False
    return df.iloc[ii, 0] != '•' and df.iloc[ii, 0] != '' and is_indicator(ii + 1, df)

def is_year(ii, df):
    return 'Overall' in df.iloc[ii, -1]

def get_year_dict(ii, df):
    if is_year(ii, df):
        year_dict = {}
        for jj, year in enumerate(df.iloc[ii]):
            if ('20' in year) and ('Overall' not in year):
                year_dict[year[:4]] = jj
        return year_dict
    else:
        raise ValueError(f"Row {ii} does not contain years.")

def is_indicator(ii, df):
    return df.iloc[ii, 0] == '•'

def is_subindicator(ii, df):
    return is_data(ii, df) and df.iloc[ii, 0] == '' and df.iloc[ii, 1] != ''

def is_data(ii, df):
    return np.any([df.iloc[ii, jj] not in ['', 'N'] for jj in range(2, 11)])

def format_chars(label):
    # >= and <= not saved properly in csv
    # get rid of '•' and trailing whitespace
    if '≥' in label:
        label = label.replace('≥', '>=')
    if '≤' in label:
        label = label.replace('≤', '<=')
    if '•' in label:
        label = label.replace('•', '')
    return label.rstrip()

def get_name(filename):
    name = ''
    for word in filename.split('-')[:-2]:
        name += word.capitalize()
    return name

In [ ]:
df = pd.DataFrame()
filenames = os.listdir('../data/PRAMS/pdf/')
for filename in filenames:
    if ('2016' in filename) and ('All' not in filename):
        df_state = pdf_to_df('../data/PRAMS/pdf/' + filename)
        state_name = get_name(filename)
        df_state['State'] = state_name
        df = pd.concat([df, df_state])
        print(f"State: {state_name}, Rows: {len(df_state)}")
df.to_csv('../data/PRAMS/csv/PRAMS_2016_2019.csv', index=False)